In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

PATH = "model_VGG16.pt"

In [3]:
# Specify transforms using torchvision.transforms as transforms library
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
TRAIN_DATA_PATH = "./images/train/"
TEST_DATA_PATH = "./images/test"
BATCH_SIZE = 32


# Load in each dataset and apply transformations using the torchvision.datasets as datasets library
train_set = datasets.ImageFolder(TRAIN_DATA_PATH, transform=transformations)
test_set = datasets.ImageFolder(TEST_DATA_PATH, transform=transformations)

# Put into a Dataloader using torch library
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [7]:
# Get pretrained model using torchvision.models as models library
model = models.vgg16(pretrained=True)

# Turn off training for their parameters
for param in model.parameters():
    param.requires_grad = False
    
# Create new classifier for model using torch.nn as nn library
classifier_input = model.classifier[6].in_features              # VGG classifier is 6-layer array; we replace last entry
num_labels = 9

classifier = nn.Sequential(nn.Linear(classifier_input, 1024),
                           nn.ReLU(),
                           nn.Linear(1024, 512),
                           nn.ReLU(),
                           nn.Linear(512, num_labels),
                           nn.LogSoftmax(dim=1))

# Replace default classifier with new classifier
model.classifier[6] = classifier

In [8]:
# Find the device available to use using torch library
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device specified above
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
# Set the error function using torch.nn as nn library
criterion = nn.NLLLoss()

# Set the optimizer function using torch.optim as optim library
optimizer = optim.Adam(model.classifier.parameters())

epochs = 10
for epoch in range(epochs):
    train_loss = 0
    test_loss = 0
    accuracy = 0
    
    # Training the model
    model.train()
    counter = 0
    for inputs, labels in train_loader:
        # Move to device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Clear optimizers
        optimizer.zero_grad()
        
        # Forward pass 
        output = model.forward(inputs)
        
        # Loss
        loss = criterion(output, labels)
        
        # Calculate gradients (backpropagation)
        loss.backward()
        
        # Adjust parameters based on gradients
        optimizer.step()
        
        # Add the loss to the training set's running loss
        train_loss += loss.item()*inputs.size(0)
        
        # Print the progress of our training
        counter += 1
        print(counter, "/", len(train_loader))
    
    # Evaluating the model
    model.eval()
    counter = 0
    # Tell torch not to calculate gradients
    with torch.no_grad():
        for inputs, labels in test_loader:
            # Move to device
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            output = model.forward(inputs)
            
            # Calculate loss
            testloss = criterion(output, labels)
            
            # Add loss to test set's running loss
            test_loss += testloss.item()*inputs.size(0)
            
            # Since our model outputs a LogSoftmax, find the real percentages by reversing the log function
            output = torch.exp(output)
            
            # Get the top class of the output
            top_p, top_class = output.topk(1, dim=1)
            
            # See how many of the classes were correct
            equals = top_class == labels.view(*top_class.shape)
            
            # Calculate the mean (get the accuracy for this batch) and add it to the running accuracy for this epoch
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            # Print the progress of our evaluation
            counter += 1
            print(counter, "/", len(test_loader))
    
    # Get the average loss for the entire epoch
    train_loss = train_loss/len(train_loader.dataset)
    test_loss = test_loss/len(test_loader.dataset)
    
    # Print out the information
    print('Accuracy: ', accuracy / len(test_loader))
    print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f}'.format(epoch, train_loss, test_loss))

C:\Users\Varun Sridhar\Anaconda3\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


1 / 112
2 / 112
3 / 112
4 / 112
5 / 112
6 / 112
7 / 112
8 / 112
9 / 112
10 / 112
11 / 112
12 / 112
13 / 112
14 / 112
15 / 112
16 / 112
17 / 112
18 / 112
19 / 112
20 / 112
21 / 112
22 / 112
23 / 112
24 / 112
25 / 112
26 / 112
27 / 112
28 / 112
29 / 112
30 / 112
31 / 112
32 / 112
33 / 112
34 / 112
35 / 112
36 / 112
37 / 112
38 / 112
39 / 112
40 / 112
41 / 112
42 / 112
43 / 112
44 / 112
45 / 112
46 / 112
47 / 112
48 / 112
49 / 112
50 / 112
51 / 112
52 / 112
53 / 112
54 / 112
55 / 112
56 / 112
57 / 112
58 / 112
59 / 112
60 / 112
61 / 112
62 / 112
63 / 112
64 / 112
65 / 112
66 / 112
67 / 112
68 / 112
69 / 112
70 / 112
71 / 112
72 / 112
73 / 112
74 / 112
75 / 112
76 / 112
77 / 112
78 / 112
79 / 112
80 / 112
81 / 112
82 / 112
83 / 112
84 / 112
85 / 112
86 / 112
87 / 112
88 / 112
89 / 112
90 / 112
91 / 112
92 / 112
93 / 112
94 / 112
95 / 112
96 / 112
97 / 112
98 / 112
99 / 112
100 / 112
101 / 112
102 / 112
103 / 112
104 / 112
105 / 112
106 / 112
107 / 112
108 / 112
109 / 112
110 / 112
111 / 11

37 / 112
38 / 112
39 / 112
40 / 112
41 / 112
42 / 112
43 / 112
44 / 112
45 / 112
46 / 112
47 / 112
48 / 112
49 / 112
50 / 112
51 / 112
52 / 112
53 / 112
54 / 112
55 / 112
56 / 112
57 / 112
58 / 112
59 / 112
60 / 112
61 / 112
62 / 112
63 / 112
64 / 112
65 / 112
66 / 112
67 / 112
68 / 112
69 / 112
70 / 112
71 / 112
72 / 112
73 / 112
74 / 112
75 / 112
76 / 112
77 / 112
78 / 112
79 / 112
80 / 112
81 / 112
82 / 112
83 / 112
84 / 112
85 / 112
86 / 112
87 / 112
88 / 112
89 / 112
90 / 112
91 / 112
92 / 112
93 / 112
94 / 112
95 / 112
96 / 112
97 / 112
98 / 112
99 / 112
100 / 112
101 / 112
102 / 112
103 / 112
104 / 112
105 / 112
106 / 112
107 / 112
108 / 112
109 / 112
110 / 112
111 / 112
112 / 112
1 / 28
2 / 28
3 / 28
4 / 28
5 / 28
6 / 28
7 / 28
8 / 28
9 / 28
10 / 28
11 / 28
12 / 28
13 / 28
14 / 28
15 / 28
16 / 28
17 / 28
18 / 28
19 / 28
20 / 28
21 / 28
22 / 28
23 / 28
24 / 28
25 / 28
26 / 28
27 / 28
28 / 28
Accuracy:  0.9523809530905315
Epoch: 6 	Training Loss: 0.162728 	Test Loss: 0.142901
1 /

In [10]:
torch.save(model, PATH)

In [11]:
model = torch.load(PATH)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1